# Manual Neuron Split tutorial

### Vivado container bug fix

In [ ]:
import os

os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libudev.so.1'
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
#os.environ['LM_LICENSE_FILE'] = 'XXXX@your.xilinx.licence.server' or filepath

## MNIST Example Model

### Load the MNIST dataset

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from PIL import Image

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# One-hot encode the labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Flatten the images and/or resize
train_images = np.array([np.array(Image.fromarray(img).resize((16, 16))).flatten() for img in train_images])
test_images = np.array([np.array(Image.fromarray(img).resize((16, 16))).flatten() for img in test_images])

# Normalize the images to the range [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

np.save('X_train_val.npy', train_images)
np.save('X_test.npy', test_images)
np.save('y_train_val.npy', train_labels)
np.save('y_test.npy', test_labels)

### Create and train the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.regularizers import l1
from keras.optimizers import Adam

# Create & train model
model = Sequential()
model.add(
    Dense(
        46,
        input_shape=(256,),
        name='fc1',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(
    Dense(
        10,
        name='output',
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(Activation(activation='softmax', name='softmax'))

# Compile the model
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if labels are integers
    metrics=['accuracy']
)
    
# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.2)

## Manual Neuron Split

In [ ]:
import hls4ml
import keras

# list of sub models
sub_models = []

# Splitting layer 1 in 2 parallel sub_models
# Get the original layer weights and biases
original_layer = model.get_layer('fc1')
weights, biases = original_layer.get_weights()

# Split the weights and biases
split_index = weights.shape[1] // 2
weights1_1 = weights[:, :split_index]
weights1_2 = weights[:, split_index:] 
biases1_1 = biases[:split_index]
biases1_2 = biases[split_index:]

# Create a new sub_model for the first half
sub_model1 = Sequential()
sub_model1.add(
    Dense(
            23,
            input_shape=(256,),
            name='fc1_1',
            kernel_initializer='lecun_uniform',
            kernel_regularizer=l1(0.0001)
        )
)
# Set the weights
sub_model1.layers[0].set_weights([weights1_1, biases1_1])
sub_models.append(sub_model1)


# Create a new sub_model for the second half
sub_model2 = Sequential()
sub_model2.add(
    Dense(
            23,
            input_shape=(256,),
            name='fc1_2',
            kernel_initializer='lecun_uniform',
            kernel_regularizer=l1(0.0001)
        )
)
# set the second half
sub_model2.layers[0].set_weights([weights1_2, biases1_2])
sub_models.append(sub_model2)

# Create a new sub_model with the remaining layer
sub_model3 = keras.Sequential([keras.layers.InputLayer(model.layers[1].input_shape[1:]), model.layers[1], model.layers[2]])
sub_models.append(sub_model3)

sub_model1.build()
sub_model2.build()
sub_model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Evaluate the sub_models
x1 = sub_model1.predict(test_images)
x2 = sub_model2.predict(test_images)
X_test = np.concatenate((x1, x2), axis=1)
sub_model3.evaluate(X_test, test_labels, batch_size=32)